# **Importing Dependencies**

In [ ]:
#Importing Dependancies
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Defining a function to do necessary preprocessing
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        try:
            img = Image.open(os.path.join(directory, filename))
            img = img.resize((128, 128))
            img = img.convert('RGB')
            img = np.array(img) / 255.0
            images.append(img)
        except OSError as e:
            print(f"Error loading {os.path.join(directory, filename)}: {e}")
            continue
    return images

# **Importing Data**

In [ ]:
#Loading Data
fr_test = load_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/fractured')
fr_train = load_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/fractured')
fr_val = load_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val/fractured')
no_fr_test = load_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/not fractured')
no_fr_train = load_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured')
no_fr_val = load_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val/not fractured')

# **Visualization**

In [ ]:
# Visuals of pictures we are working on.
fig, axes = plt.subplots(1, 6, figsize=(20, 10))
axes[0].imshow(fr_test[0])
axes[1].imshow(fr_train[0])
axes[2].imshow(fr_val[0])
axes[3].imshow(no_fr_test[0])
axes[4].imshow(no_fr_train[0])
axes[5].imshow(no_fr_val[0])
axes[0].set_title('Fractured')
axes[1].set_title('Fractured')
axes[2].set_title('Fractured')
axes[3].set_title('Not Fractured')
axes[4].set_title('Not Fractured')
axes[5].set_title('Not Fractured')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
#Combining Data for Convinience 
train_images = np.concatenate((fr_train, no_fr_train))
val_images = np.concatenate((fr_val, no_fr_val))
test_images = np.concatenate((fr_test, no_fr_test))

**Assigning Class Labels**

In [ ]:
#Assigning Labels 
train_labels = np.concatenate((np.ones(len(fr_train)), np.zeros(len(no_fr_train))))
val_labels = np.concatenate((np.ones(len(fr_val)), np.zeros(len(no_fr_val))))
test_labels = np.concatenate((np.ones(len(fr_test)), np.zeros(len(no_fr_test))))

In [ ]:
train_labels

# **Tensorflow Input Data Pipeline**

In [ ]:
#Tensorflow Input Data Pipeline
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [ ]:
#Batching
batch_size = 32
train = train_ds.shuffle(buffer_size=len(train_images)).batch(batch_size)

In [ ]:
val = val_ds.batch(batch_size)
test= test_ds.batch(batch_size)

In [ ]:
train.as_numpy_iterator().next()

# **Model Architecture**

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(128, 128, 3), padding='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, validation_data = val, epochs = 10, verbose = 1)

# **Model Evaluation**

In [ ]:
model.evaluate(test)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Accuracy', 'Val Accuracy'], loc = 'upper right')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'Val Loss'], loc = 'upper right')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.show()

# **Confusion and Classification Metrics**

In [ ]:
pred = model.predict(test)
pred = (pred > 0.5).astype('int')
    
pred

In [ ]:
# We have data in batches so, make truth data to match the shape of Predicted. You can do it in many ways.
truth = np.concatenate([y for _, y in test], axis=0)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix



print('Classification Report: \n', classification_report(truth, pred))

In [ ]:
cm = confusion_matrix(truth, pred)

In [ ]:
import seaborn as sns

total_per_class = cm.sum(axis=1)
percentage_matrix = (cm / total_per_class[:, None]) * 100

sns.heatmap(percentage_matrix, annot=True, fmt=".1f")

#                                                                      **Thank You**